В этом ноутбуке мы протестируем каждую из двух моделей  RoBERTa. Одна обучена на spam sms, другая на spam emails. Протестируем каждую модель на каждом из датасетов (spam sms dataset и spam emails dataset). 

In [1]:
model_name = 'roberta'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
from transformers import RobertaTokenizerFast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.2 MB/s eta 0:00:00


In [4]:
!pip install tensorflow-text
import tensorflow_text as text

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.models import load_model
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
spam_sms = 'spam sms'
spam_emails = 'spam emails'
roberta_trained_on_spam_sms = 'roberta trained on spam sms'
roberta_trained_on_spam_emails = 'roberta trained on spam emails'

In [6]:
models_dict = dict()
models_dict[roberta_trained_on_spam_sms] = load_model('/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_sms_20_january')
models_dict[roberta_trained_on_spam_emails] = load_model('/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_emails_20_january')

In [7]:
path_to_spam_sms_dataset = '/content/drive/MyDrive/data_for_colab/spam_sms.csv'
path_to_spam_emails_dataset = '/content/drive/MyDrive/data_for_colab/spam_emails.csv'

In [8]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [10]:
MAX_LEN = 128
def tokenize_roberta(data,max_len=MAX_LEN) :
  input_ids = []
  attention_masks = []
  for i in range(len(data)):
      encoded = tokenizer_roberta.encode_plus(
          data[i],
          add_special_tokens=True,
          max_length=max_len,
          padding='max_length',
          return_attention_mask=True
      )
      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [11]:
def test_model_on_dataset_and_write_results_to_dataframe(model_name=None, dataset_name=None, dataframe_to_write_answer=None):
    global spam_sms, spam_emails
    global roberta_on_spam_sms
    global roberta_on_spam_emails
    global models_dict
    global path_to_hotel_reviews_dataset
    global path_to_movie_reviews_dataset

    if model_name is None or dataset_name is None or dataframe_to_write_answer is None:
        raise ValueError("Wrong arguments passed to function: there are none arguments!")
    if model_name != roberta_trained_on_spam_sms and model_name != roberta_trained_on_spam_emails:
        raise ValueError("Wrong model_name!")
    if dataset_name != spam_sms and dataset_name != spam_emails:
        raise ValueError("Wrong dataset_name!")

    model = models_dict[model_name] # загрузили пользователем заданную модель 
    # (она уже обучена на определенном датасете)

    parameter_const = 1.8 # подобрали

    if dataset_name == spam_sms: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_sms_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
        df.columns = ['IS_SPAM', 'DATA_COLUMN']
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])
        df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    elif dataset_name == spam_emails: # if you edit this line, edit the next line too!
        df = pd.read_csv(path_to_spam_emails_dataset, encoding = "ISO-8859-1")
        df.drop(columns=['Unnamed: 0', 'label'], inplace=True)
        df.columns = ['DATA_COLUMN', 'IS_SPAM']
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x.replace('\r\n', ' ').replace('\n', ' '))
        df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])
        df_positive = df[df['IS_SPAM']==1]
        df_negative = df[df['IS_SPAM']==0]
        # Тестовая выборка
        n_test = df_negative.shape[0] // 2
        df_negative_test = df_negative.tail(n_test)
        n_test = df_positive.shape[0] // 2
        df_positive_test = df_positive.tail(n_test)
        df_balanced_test = pd.concat([df_negative_test, df_positive_test])

    X_test = df_balanced_test['DATA_COLUMN'].squeeze().values
    y_test = df_balanced_test['IS_SPAM'].squeeze().values

    test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)
    result_roberta = model.predict([test_input_ids,test_attention_masks])
    y_predicted = np.where(result_roberta > 0.5, 1, 0)

    row_name_in_dataframe_to_write_answer = model_name + ' tested on ' + dataset_name + ' dataset'

    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'accuracy'] = accuracy_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'precision'] = precision_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'recall'] = recall_score(y_test, y_predicted)
    dataframe_to_write_answer.loc[row_name_in_dataframe_to_write_answer, 'f1_score'] =  f1_score(y_test, y_predicted)
    print(model_name, ' on dataset', dataset_name, 'result', accuracy_score(y_test, y_predicted), precision_score(y_test, y_predicted), recall_score(y_test, y_predicted), f1_score(y_test, y_predicted) )

In [12]:
answer_dataframe = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [13]:
for cur_model_name in [roberta_trained_on_spam_sms, roberta_trained_on_spam_emails]:
    for cur_dataset_name in [spam_sms, spam_emails]:
        test_model_on_dataset_and_write_results_to_dataframe(cur_model_name, cur_dataset_name, answer_dataframe)

88/88 [==============================] - 28s 257ms/step
roberta trained on spam sms  on dataset spam sms result 0.99245960502693 0.9705882352941176 0.9731903485254692 0.9718875502008032
81/81 [==============================] - 21s 261ms/step
roberta trained on spam sms  on dataset spam emails result 0.523017408123791 0.3207407407407407 0.5781041388518025 0.41257741781800855
88/88 [==============================] - 24s 257ms/step
roberta trained on spam emails  on dataset spam sms result 0.5324955116696589 0.2005157962604771 0.8337801608579088 0.3232848232848233
81/81 [==============================] - 21s 258ms/step
roberta trained on spam emails  on dataset spam emails result 0.9771760154738878 0.9831932773109243 0.9372496662216289 0.959671907040328


In [14]:
answer_dataframe


,accuracy,precision,recall,f1_score
roberta trained on spam sms tested on spam sms dataset,0.99246,0.970588,0.97319,0.971888
roberta trained on spam sms tested on spam emails dataset,0.523017,0.320741,0.578104,0.412577
roberta trained on spam emails tested on spam sms dataset,0.532496,0.200516,0.83378,0.323285
roberta trained on spam emails tested on spam emails dataset,0.977176,0.983193,0.93725,0.959672


Сохраняем dataframe

In [15]:
name_for_dataframe = model_name + ' on spam testing result'

In [16]:
answer_dataframe.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_dataframe + '.csv')